<a href="https://colab.research.google.com/github/davidemichelon11/DL_Assignment/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [8]:
from google.colab import drive
import os
import shutil
from tqdm import tqdm
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.tensorboard import SummaryWriter

# Extract data and create dataset

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!unzip -q -o gdrive/MyDrive/Adaptiope.zip 

In [4]:
classes = ["/backpack", "/bookcase", "/car jack", "/comb", "/crown", "/file cabinet", "/flat iron", "/game controller", "/glasses",
           "/helicopter", "/ice skates", "/letter tray", "/monitor", "/mug", "/network switch", "/over-ear headphones", "/pen",
           "/purse", "/stand mixer", "/stroller"]

for d, td in zip(["Adaptiope/product_images", "Adaptiope/real_life"], ["adaptiope_small/product_images", "adaptiope_small/real_life"]):
  os.makedirs(td)
  for c in tqdm(classes):
    c_path = ''.join((d, c))
    c_target = ''.join((td, c))
    shutil.copytree(c_path, c_target)

100%|██████████| 20/20 [00:05<00:00,  3.76it/s]


# Create dataloader

In [2]:
def get_data(batch_size, product_root, real_root):
  # resizing and cropping
  # prepare data transformations for the train loader
  transform = list()
  transform.append(T.Resize((256, 256)))                      # resize each PIL image to 256 x 256
  transform.append(T.RandomCrop((224, 224)))                 # randomly crop a 224 x 224 patch
  transform.append(T.ToTensor())                              # convert Numpy to Pytorch Tensor
  transform.append(T.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225]))    # normalize with ImageNet mean
  transform = T.Compose(transform)                            # compose the above transformations into one
    
  # load data
  dataset_prod = torchvision.datasets.ImageFolder(root=product_root, transform=transform)
  dataset_real = torchvision.datasets.ImageFolder(root=real_root, transform=transform)
  
  # create train and test splits (80/20)
  num_samples = len(dataset_prod) # same number of samples in this dataset
  training_samples = int(num_samples * 0.8 + 1)
  test_samples = num_samples - training_samples

  train_data_prod, test_data_prod = torch.utils.data.random_split(dataset_prod, [training_samples, test_samples])
  train_data_real, test_data_real = torch.utils.data.random_split(dataset_real, [training_samples, test_samples])

  # initialize dataloaders
  train_loader_prod = torch.utils.data.DataLoader(train_data_prod, batch_size, shuffle=True)
  test_loader_prod = torch.utils.data.DataLoader(test_data_prod, batch_size, shuffle=False)
  
  train_loader_real = torch.utils.data.DataLoader(train_data_real, batch_size, shuffle=True)
  test_loader_real = torch.utils.data.DataLoader(test_data_real, batch_size, shuffle=False)
  
  return (train_loader_prod, test_loader_prod), (train_loader_real, test_loader_real)

# Create model

In [3]:
def initialize_resnet(num_classes):

  # load the pre-trained Alexnet
  resnet = torchvision.models.resnet50(pretrained=True)
  
  # get the number of neurons in the second last layer
  in_features = resnet.fc.in_features
  
  # re-initalize the output layer
  resnet.fc = torch.nn.Linear(in_features=in_features, 
                                          out_features=num_classes)
  
  return resnet

# Specify cost function and optimizer

In [4]:
def get_cost_function():
  cost_function = torch.nn.CrossEntropyLoss()
  return cost_function

In [5]:
def get_optimizer(model, lr, wd, momentum):
  
  # we will create two groups of weights, one for the newly initialized layer
  # and the other for rest of the layers of the network
  
  final_layer_weights = []
  rest_of_the_net_weights = []
  
  # iterate through the layers of the network
  for name, param in model.named_parameters():
    if name.startswith('fc'):
      final_layer_weights.append(param)
    else:
      rest_of_the_net_weights.append(param)
  
  # assign the distinct learning rates to each group of parameters
  optimizer = torch.optim.SGD([
      {'params': rest_of_the_net_weights},
      {'params': final_layer_weights, 'lr': lr}
  ], lr=lr/10, weight_decay=wd, momentum=momentum)
  
  return optimizer

# Train model

In [6]:
def training_step(model, source_train_loader, target_train_loader, optimizer, 
                  cost_function, device='cuda:0'):
  source_samples = 0.
  target_samples = 0.
  cumulative_ce_loss = 0.
  cumulative_accuracy = 0.
  
  target_iter = iter(target_train_loader)

  # strictly needed if network contains layers which has different behaviours between train and test
  model.train()
  for batch_idx, (inputs_source, targets) in enumerate(tqdm(source_train_loader)):
    
    # get target data. If the target iterator reaches the end, restart it
    try:
      inputs_target, _ = next(target_iter)
    except:
      target_iter = iter(target_train_loader)
      inputs_target, _ = next(target_iter)
    
    inputs = torch.cat((inputs_source, inputs_target), dim=0)
    
    # load data into GPU
    inputs = inputs.to(device)
    targets = targets.to(device)
      
    # forward pass
    outputs = model(inputs)
    
    # split the source and target outputs
    source_output, target_output = torch.split(outputs, 
                                               split_size_or_sections=outputs.shape[0] // 2, 
                                               dim=0)
    
    # apply the losses
    ce_loss = cost_function(source_output, targets)
    
    loss = ce_loss # later we will add other losses
    
    # backward pass
    loss.backward()
    
    # update parameters
    optimizer.step()
    
    # reset the optimizer
    optimizer.zero_grad()

    # print statistics
    source_samples += inputs_source.shape[0]
    target_samples += inputs_target.shape[0]
    
    cumulative_ce_loss += ce_loss.item()
    _, predicted = source_output.max(1)
    cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_ce_loss/source_samples, cumulative_accuracy/source_samples*100


def test_step(model, target_test_loader, cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  # strictly needed if network contains layers which has different behaviours between train and test
  model.eval()

  with torch.no_grad():

    for batch_idx, (inputs, targets) in enumerate(tqdm(target_test_loader)):

      # load data into GPU
      inputs = inputs.to(device)
      targets = targets.to(device)
        
      # forward pass
      outputs = model(inputs)

      # apply the loss
      loss = cost_function(outputs, targets)

      # print statistics
      samples += inputs.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = outputs.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100

# Execute everything

In [ ]:
batch_size = 20
device='cuda:0'
learning_rate=0.01
weight_decay=0.000001
momentum=0.9
epochs=50

prod_root = 'adaptiope_small/product_images'
real_root = 'adaptiope_small/real_life'

dataloaders_prod, dataloaders_real = get_data(batch_size, prod_root, real_root)
train_loader_prod, test_loader_prod = dataloaders_real
train_loader_real, test_loader_real = dataloaders_prod

num_classes = len(set(train_loader_prod.dataset.dataset.targets))

ResNet = initialize_resnet(num_classes).to(device)

optimizer = get_optimizer(ResNet, learning_rate, weight_decay, momentum)

cost_function = get_cost_function()

for e in range(epochs):
  print('Epoch: {}/{}'.format(e+1, epochs))
  train_ce_loss, train_accuracy = training_step(model=ResNet,
                                                source_train_loader=train_loader_prod,
                                                target_train_loader=train_loader_real,
                                                optimizer=optimizer, 
                                                cost_function=cost_function,
                                                device=device)
  
  test_loss, test_accuracy = test_step(model=ResNet, 
                                       target_test_loader=test_loader_real, 
                                       cost_function=cost_function, 
                                       device=device)
  
  print('Train: CE loss {:.5f}, Accuracy {:.2f}'.format(train_ce_loss, train_accuracy))
  print('Test: CE loss {:.5f}, Accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')